Todo:
- vyplnit nulls pro narození nejpřesnějším známým datem

In [2]:
import os
import polars as pl

In [3]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [4]:
df = pl.read_ndjson(os.path.join('data_raw/poslanectvo','*.ndjson'), schema_overrides={'navrhovatel':pl.String,'kandidatka':pl.String})

In [5]:
df.sample(100)

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni,clenstvi
str,str,str,str,str,str,str,str,str,str,str,list[struct[4]]
"""6230""","""7""","""6230_7.html""","""Stanislav Pfléger""","""Ústecký""","""m""","""26. 10. 2013""","""26. 10. 2017""","""ANO2011""",null,"""1958""","[{""člen"",""Hospodářský výbor"",""4. 12. 2013"",""26. 10. 2017""}, {""člen"",""Kontrolní výbor"",""4. 12. 2013"",""26. 10. 2017""}, … {""člen"",""Dozorčí rada Státního fondu dopravní infrastruktury"",""14. 5. 2014"",""14. 5. 2018""}]"
"""67""","""2""","""67_2.html""","""Ing. Jan Klas""","""Středočeský""","""m""","""1. 6. 1996""","""19. 6. 1998""","""ODS""",null,"""1958""","[{""člen"",""Výbor pro obranu a bezpečnost"",""2. 7. 1996"",""19. 6. 1998""}, {""místopředsed"",""Výbor pro obranu a bezpečnost"",""3. 7. 1996"",""19. 6. 1998""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""11. 6. 1996"",""19. 6. 1998""}]"
"""6734""","""9""","""6734_9.html""","""Ing. Petr Liška""","""Ústecký""","""m""","""9. 10. 2021""",null,"""PirSTAN""","""STAN""","""16. 3. 1970""","[{""člen"",""Výbor pro veřejnou správu a regionální rozvoj"",""23. 11. 2021"",null}, {""člen"",""Výbor pro obranu"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Tanzanie"",""5. 12. 2022"",null}]"
"""6436""","""9""","""6436_9.html""","""Ing. Petr Beitl""","""Liberecký""","""m""","""9. 10. 2021""",null,"""SPOLU""","""ODS""","""5. 12. 1966""","[{""člen"",""Zahraniční výbor"",""23. 11. 2021"",null}, {""člen"",""Výbor pro evropské záležitosti"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Paraguyaská republika"",""28. 3. 2024"",null}]"
"""6179""","""8""","""6179_8.html""","""Mgr. Ondřej Benešík""","""Zlínský""","""m""","""21. 10. 2017""","""20. 10. 2021""","""KDU-ČSL""",null,"""1976""","[{""člen"",""Výbor pro evropské záležitosti"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Zemědělský výbor"",""28. 11. 2017"",""15. 12. 2017""}, … {""člen"",""Skupina ČR - Chorvatsko"",""18. 4. 2018"",""20. 10. 2021""}]"
"""5301""","""6""","""5301_6.html""","""Mgr. David Šeich""","""Jihomoravský""","""m""","""29. 5. 2010""","""28. 8. 2013""","""ODS""",null,"""1976""","[{""člen"",""Zahraniční výbor"",""7. 7. 2010"",""28. 8. 2013""}, {""místopředsed"",""Zahraniční výbor"",""8. 7. 2010"",""28. 8. 2013""}, … {""člen"",""Skupina ČR - státy jihovýchodní Evropy (Albánie, Bosna a Hercegovina, Černá Hora, Kosovo, Makedonie/FYROM, Moldavie, Rumunsko)"",""14. 12. 2010"",""28. 8. 2013""}]"
"""5273""","""7""","""5273_7.html""","""JUDr. Jiří Pospíšil""","""Plzeňský""","""m""","""26. 10. 2013""","""20. 6. 2014""","""ODS""",null,"""1975""","[{""člen"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""4. 12. 2013"",""26. 3. 2014""}, {""člen"",""Ústavně právní výbor"",""4. 12. 2013"",""26. 3. 2014""}, … {""člen"",""Nezařazení"",""4. 2. 2014"",""20. 6. 2014""}]"
"""6747""","""9""","""6747_9.html""","""Martin Exner""","""Středočeský""","""m""","""9. 10. 2021""",null,"""PirSTAN""","""STAN""","""20. 12. 1963""","[{""člen"",""Výbor pro evropské záležitosti"",""23. 11. 2021"",null}, {""člen"",""Výbor pro bezpečnost"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Alžírsko"",""14. 11. 2023"",null}]"
"""5965""","""8""","""5965_8.html""","""Mgr. Roman Sklenák""","""Jihomoravský""","""m""","""31. 3. 2018""","""17. 9. 2021""","""ČSSD""",null,"""1970""","[{""člen"",""Výbor pro životní prostředí"",""11. 4. 2018"",""7. 12. 2018""}, {""člen"",""Výbor pro sociální politiku"",""11. 4. 2018"",""17. 9. 2021""}, … {""člen"",""Dozorčí rada VZP"",""24. 1. 2018"",""24. 9. 2021""}]"


In [6]:
obdobi = {
    '9':'2021',
    '8':'2017',
    '7':'2013',
    '6':'2010',
    '5':'2006',
    '4':'2002',
    '3':'1998',
    '2':'1996',
    '1':'1992'
}

In [7]:
def najdi_tituly(jmeno):
    jmeno = jmeno.replace('ml.','mladší').replace('st.','starší')
    tituly = []
    za = None
    if ',' in jmeno:
        tituly_za = ' '.join(jmeno.split(',')[1:])
        za = [t.strip() for t in tituly_za.strip().split(' ') if t != '']
        jmeno = jmeno.split(',')[0]
    pred = [t for t in jmeno.split(' ') if '.' in t]
    jmeno = jmeno.split('.')[-1].strip()
    if za:
        tituly = pred + za
    else:
        tituly = pred
    return {'jmeno_ciste':jmeno,'tituly':tituly}

In [8]:
najdi_tituly("PhDr. Walter Bartoš, Ph.D., MBA")

{'jmeno_ciste': 'Walter Bartoš', 'tituly': ['PhDr.', 'Ph.D.', 'MBA']}

In [9]:
len(df)

1606

In [10]:
df.filter(pl.col('jmeno').str.contains('Benda'))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni,clenstvi
str,str,str,str,str,str,str,str,str,str,str,list[struct[4]]
"""4""","""1""","""4_1.html""","""Marek Benda""","""Středočeský""","""m""","""6. 6. 1992""","""6. 6. 1996""",null,null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Ústavně právní výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""1. 4. 1996"",""6. 6. 1996""}]"
"""4""","""2""","""4_2.html""","""Marek Benda""","""Severomoravský""","""m""","""1. 6. 1996""","""19. 6. 1998""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""27. 6. 1996"",""19. 6. 1998""}, {""člen"",""Ústavně právní výbor"",""2. 7. 1996"",""19. 6. 1998""}, … {""místopředsed"",""Poslanecký klub Občanské demokratické strany"",""16. 7. 1996"",""9. 6. 1997""}]"
"""4""","""3""","""4_3.html""","""Marek Benda""","""Severomoravský""","""m""","""20. 6. 1998""","""20. 6. 2002""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""15. 7. 1998"",""20. 6. 2002""}, {""člen"",""Ústavně právní výbor"",""22. 7. 1998"",""20. 6. 2002""}, … {""člen"",""Skupina ČR - Tunisko"",""1. 4. 1999"",""20. 6. 2002""}]"
"""4""","""4""","""4_4.html""","""Marek Benda""","""Hlavní""","""m""","""20. 7. 2004""","""15. 6. 2006""","""ODS""",null,"""1968""","[{""člen"",""Ústavně právní výbor"",""24. 9. 2004"",""15. 6. 2006""}, {""člen"",""Podvýbor pro kriminalitu mládeže"",""31. 3. 2005"",""15. 6. 2006""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""20. 7. 2004"",""15. 6. 2006""}]"
"""4""","""5""","""4_5.html""","""Marek Benda""","""Hlavní""","""m""","""3. 6. 2006""","""3. 6. 2010""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""29. 6. 2006"",""3. 6. 2010""}, {""místopředsed"",""Mandátový a imunitní výbor"",""29. 6. 2006"",""15. 9. 2006""}, … {""člen"",""Skupina ČR - Maďarsko"",""7. 5. 2008"",""3. 6. 2010""}]"
"""4""","""6""","""4_6.html""","""Marek Benda""","""Hlavní""","""m""","""29. 5. 2010""","""28. 8. 2013""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""24. 6. 2010"",""28. 8. 2013""}, {""člen"",""Ústavně právní výbor"",""7. 7. 2010"",""28. 8. 2013""}, … {""člen"",""Skupina ČR - Arménie"",""9. 5. 2012"",""28. 8. 2013""}]"
"""4""","""7""","""4_7.html""","""Marek Benda""","""Hlavní""","""m""","""26. 10. 2013""","""26. 10. 2017""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""27. 11. 2013"",""12. 12. 2013""}, {""předsed"",""Mandátový a imunitní výbor"",""27. 11. 2013"",""12. 12. 2013""}, … {""člen"",""Skupina ČR - Rumunsko, Moldavská republika"",""26. 11. 2014"",""26. 10. 2017""}]"
"""4""","""8""","""4_8.html""","""Marek Benda""","""Hlavní""","""m""","""21. 10. 2017""","""20. 10. 2021""","""ODS""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""22. 11. 2017"",""20. 10. 2021""}, {""člen"",""Ústavně právní výbor"",""28. 11. 2017"",""20. 10. 2021""}, … {""člen"",""Skupina ČR - Izrael"",""10. 4. 2018"",""20. 10. 2021""}]"
"""4""","""9""","""4_9.html""","""Marek Benda""","""Hlavní""","""m""","""9. 10. 2021""",null,"""SPOLU""","""ODS""","""10. 11. 1968""","[{""člen"",""Mandátový a imunitní výbor"",""10. 11. 2021"",null}, {""člen"",""Zahraniční výbor"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Albánská republika"",""15. 6. 2022"",null}]"


In [11]:
df = df.with_columns(pl.col("kraj").replace("Hlavní","Praha")).with_columns(
    pl.col(["obdobi_od","obdobi_do"]).str.to_date()
).rename(
    {'narozeni':'narozeni_retezec'}
).with_columns(
    pl.col('narozeni_retezec').str.extract(r'\d{4}',0).cast(int).alias('narozeni_rok')
).with_columns(
    pl.col('narozeni_retezec').str.to_date(strict=False).alias('narozeni_datum')
).with_columns(
    pl.col('obdobi').replace(obdobi).cast(pl.Int32)
).sort(
    by=['obdobi','id']
).with_columns(
    pl.col('jmeno').map_elements(najdi_tituly,return_dtype=pl.Struct).alias('jmeno+tituly')
).unnest(
    "jmeno+tituly"
)

In [12]:
len(df)

1606

In [13]:
len(df.unique(subset=['id','jmeno']))

854

In [14]:
df.group_by("kandidatka").len()

kandidatka,len
str,u32
"""VV""",24
"""PirSTAN""",39
"""KDU-ČSL""",71
null,37
"""ČSSD""",404
"""ANO2011""",210
"""US""",15
"""KSČM""",190
"""K""",34


In [15]:
df.filter(pl.col("jmeno").str.contains("Zaorálek"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""252""",1996,"""252_2.html""","""PhDr. Lubomír Zaorálek""","""Severomoravský""","""m""",1996-06-01,1998-06-19,"""ČSSD""",null,"""1956""","[{""člen"",""Zahraniční výbor"",""2. 7. 1996"",""19. 6. 1998""}, {""místopředsed"",""Zahraniční výbor"",""4. 7. 1996"",""19. 6. 1998""}, … {""místopředsed"",""Poslanecký klub České strany sociálně demokratické"",""19. 6. 1996"",""19. 6. 1998""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"
"""252""",1998,"""252_3.html""","""PhDr. Lubomír Zaorálek""","""Severomoravský""","""m""",1998-06-20,2002-06-20,"""ČSSD""",null,"""1956""","[{""člen"",""Zahraniční výbor"",""22. 7. 1998"",""20. 6. 2002""}, {""člen"",""Organizační výbor"",""22. 7. 1998"",""20. 6. 2002""}, … {""místopředsed"",""Poslanecký klub České strany sociálně demokratické"",""24. 6. 1998"",""18. 4. 2000""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"
"""252""",2002,"""252_4.html""","""PhDr. Lubomír Zaorálek""","""Moravskoslezský""","""m""",2002-06-15,2006-06-15,"""ČSSD""",null,"""1956""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""předsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""21. 6. 2002"",""15. 6. 2006""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"
"""252""",2006,"""252_5.html""","""PhDr. Lubomír Zaorálek""","""Moravskoslezský""","""m""",2006-06-03,2010-06-03,"""ČSSD""",null,"""1956""","[{""člen"",""Organizační výbor"",""12. 9. 2006"",""3. 6. 2010""}, {""místopředsed"",""Organizační výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""5. 6. 2006"",""3. 6. 2010""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"
"""252""",2010,"""252_6.html""","""PhDr. Lubomír Zaorálek""","""Moravskoslezský""","""m""",2010-05-29,2013-08-28,"""ČSSD""",null,"""1956""","[{""člen"",""Organizační výbor"",""7. 7. 2010"",""28. 8. 2013""}, {""místopředsed"",""Organizační výbor"",""7. 7. 2010"",""28. 8. 2013""}, {""člen"",""Poslanecký klub České strany sociálně demokratické"",""4. 6. 2010"",""28. 8. 2013""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"
"""252""",2013,"""252_7.html""","""PhDr. Lubomír Zaorálek""","""Moravskoslezský""","""m""",2013-10-26,2017-10-26,"""ČSSD""",null,"""1956""","[{""člen"",""Poslanecký klub České strany sociálně demokratické"",""29. 10. 2013"",""26. 10. 2017""}, {null,""Vláda České republiky"",""29. 1. 2014"",""13. 12. 2017""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"
"""252""",2017,"""252_8.html""","""PhDr. Lubomír Zaorálek""","""Moravskoslezský""","""m""",2017-10-21,2021-10-20,"""ČSSD""",null,"""1956""","[{""člen"",""Zahraniční výbor"",""15. 12. 2017"",""27. 8. 2019""}, {""předsed"",""Zahraniční výbor"",""15. 12. 2017"",""27. 8. 2019""}, … {null,""Vláda České republiky"",""27. 8. 2019"",""17. 12. 2021""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"


In [16]:
df.group_by("navrhovatel").len()

navrhovatel,len
str,u32
"""ODS""",35
"""US""",10
"""STAN""",34
"""Piráti""",4
"""KDU-ČSL""",46
"""TOP09""",14
null,1463


In [17]:
df.group_by('obdobi').agg(pl.col('id').unique().len()).sort(by='obdobi')

obdobi,id
i32,u32
1992,37
1996,100
1998,168
2002,234
2006,220
2010,213
2013,208
2017,216
2021,210


In [18]:
df.group_by('obdobi').agg(pl.col('jmeno').unique().len()).sort(by='obdobi')

obdobi,jmeno
i32,u32
1992,37
1996,100
1998,168
2002,234
2006,220
2010,213
2013,208
2017,216
2021,210


In [19]:
najdi_tituly("RSDr. Ing. Svatomír Recman")

{'jmeno_ciste': 'Svatomír Recman', 'tituly': ['RSDr.', 'Ing.']}

In [20]:
df.group_by(['jmeno','obdobi']).len().sort(by='len',descending=True)

jmeno,obdobi,len
str,i32,u32
"""Robert Kopecký""",1998,1
"""Jana Lorencová""",2013,1
"""prof. Ing. Ivan Jáč, CSc.""",2017,1
"""Jiří Čepelka""",2006,1
"""Lukáš Kolářík""",2017,1
"""Ing. Alena Páralová""",2002,1
"""Ing. Jiří Krátký""",2006,1
"""Ing. Miroslav Máče, CSc., Ph.D""",2002,1
"""Ing. Jan Kubík""",2021,1


In [21]:
najdi_tituly("prof. Ing. Karel Rais, CSc., MBA")

{'jmeno_ciste': 'Karel Rais', 'tituly': ['prof.', 'Ing.', 'CSc.', 'MBA']}

In [22]:
df.filter(pl.col('jmeno_ciste') == '')

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]


In [23]:
df.group_by(['jmeno_ciste','obdobi']).len().sort(by='len',descending=True)

jmeno_ciste,obdobi,len
str,i32,u32
"""Rudolf Tomíček""",1996,1
"""Lukáš Pleticha""",2013,1
"""Jan Bürgermeister""",2006,1
"""Václav Frank""",1998,1
"""Leoš Heger""",2010,1
"""Lubomír Zaorálek""",2013,1
"""Jarmila Levko""",2021,1
"""Petr Sadovský""",2021,1
"""Boris Šťastný""",2010,1


In [24]:
df

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""100""",1992,"""100_1.html""","""JUDr. Hana Kordová Marvanová""","""Praha""","""f""",1992-06-06,1996-06-06,null,null,"""1962""","[{""člen"",""Ústavně právní výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""místopředsed"",""Ústavně právní výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""6. 6. 1992"",""6. 6. 1996""}]",1962,null,"""Hana Kordová Marvanová""","[""JUDr.""]"
"""105""",1992,"""105_1.html""","""JUDr. Dalibor Matulka""","""Jihomoravský""","""m""",1992-06-06,1996-06-06,null,null,"""1953""","[{""člen"",""Mandátový a imunitní výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Ústavně právní výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Komunistické strany Čech a Moravy"",""19. 1. 1994"",""6. 6. 1996""}]",1953,null,"""Dalibor Matulka""","[""JUDr.""]"
"""110""",1992,"""110_1.html""","""Mgr. Václav Nájemník""","""Východočeský""","""m""",1992-06-06,1996-06-06,null,null,"""1944""","[{""člen"",""Výbor pro veřejnou správu, regionální rozvoj a životní prostředí"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""6. 6. 1992"",""6. 6. 1996""}]",1944,null,"""Václav Nájemník""","[""Mgr.""]"
"""111""",1992,"""111_1.html""","""RNDr. Petr Nečas""","""Severomoravský""","""m""",1992-06-06,1996-06-06,null,null,"""1964""","[{""člen"",""Zahraniční výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Stálá komise pro reformu veřejné správy"",""29. 9. 1994"",""6. 6. 1996""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""6. 6. 1992"",""6. 6. 1996""}]",1964,null,"""Petr Nečas""","[""RNDr.""]"
"""12""",1992,"""12_1.html""","""Ing. František Brožík""","""Západočeský""","""m""",1992-06-06,1996-06-06,null,null,"""1955""","[{""člen"",""Hospodářský výbor"",""6. 6. 1992"",""6. 6. 1996""}, {null,""Hospodářský výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""6. 6. 1992"",""6. 6. 1996""}]",1955,null,"""František Brožík""","[""Ing.""]"
"""122""",1992,"""122_1.html""","""Ing. Hana Orgoníková""","""Východočeský""","""f""",1992-06-06,1996-06-06,null,null,"""1946""","[{""člen"",""Výbor petiční, pro lidská práva a národnosti"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Stálá komise pro práci Kanceláře Poslanecké sněmovny"",""27. 1. 1993"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""12. 10. 1994"",""6. 6. 1996""}]",1946,null,"""Hana Orgoníková""","[""Ing.""]"
"""124""",1992,"""124_1.html""","""Ing. Jaroslav Palas""","""Severomoravský""","""m""",1992-06-06,1996-06-06,null,null,"""1952""","[{""člen"",""Zemědělský výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Poslanecký klub Levého bloku"",""6. 6. 1992"",""27. 9. 1995""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""1. 11. 1995"",""6. 6. 1996""}]",1952,null,"""Jaroslav Palas""","[""Ing.""]"
"""127""",1992,"""127_1.html""","""RNDr. Jiří Payne""","""Praha""","""m""",1992-06-06,1996-06-06,null,null,"""1956""","[{""člen"",""Zahraniční výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""předsed"",""Zahraniční výbor"",""6. 6. 1992"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""6. 6. 1992"",""6. 6. 1996""}]",1956,null,"""Jiří Payne""","[""RNDr.""]"
"""136""",1992,"""136_1.html""","""RSDr. Ing. Svatomír Recman""","""Severomoravský""","""m""",1992-06-06,1996-06-06,null,null,"""1952""","[{""člen"",""Rozpočtový výbor"",""6. 6. 1992"",""6. 6. 1996""}, {""člen"",""Stálá komise pro práci Kanceláře Poslanecké sněmovny"",""27. 1. 1993"",""6. 6. 1996""}, … {""člen"",""Poslanecký klub Komunistické strany Čech a Moravy"",""15. 2. 1994"",""6. 6. 1996""}]",1952,null,"""Svatomír Recman""","[""RSDr."", ""Ing.""]"


In [25]:
df.write_parquet(os.path.join("data","mluvci.parquet"))